In [40]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.6.0


In [3]:
fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
train_images.dtype

dtype('uint8')

In [5]:
#convert dtypes to float
train_images=train_images/255.0
test_images=test_images/255.0
train_images[0].shape


(28, 28, 1)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [43]:
def build_model(hp) :
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_l_kernel',values=[3,5]),
            activation = 'relu',
            input_shape=(28,28,1)
            
        ),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
            keras.layers.BatchNormalization(axis=-1),
        keras.layers.Conv2D(
             filters=hp.Int('conv_2_filter',min_value=32,max_value=64, step =16),
             kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
            activation = 'relu'
        ),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.BatchNormalization(axis=-1),
                                        
                                        
        keras.layers.Flatten(),
        keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', 
                            values=[1e-2,1e-3])),
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
    
    return model
        
    

In [35]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [57]:
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=1, directory='output1',project_name='Mnist_Fashion')

In [58]:
# execute in GPU
tuner_search.search(train_images,train_labels, epochs=1,validation_split=0.1)

Trial 1 Complete [00h 01m 14s]
val_accuracy: 0.7778333425521851

Best val_accuracy So Far: 0.7778333425521851
Total elapsed time: 00h 01m 14s
INFO:tensorflow:Oracle triggered exit


In [59]:
model =tuner_search.get_best_models(num_models=1)[0]
#model.summary()


In [61]:
#model.fit(train_images, train_labels, epochs=5,validation_split=0.1, initial_epoch=3)

In [ ]:
pred = model.predict_classes(test_images)
#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(test_labels, pred)*100)